In [27]:
import pickle
from sklearn.preprocessing import LabelEncoder

# Categories to encode
CATEGORIES = ["Cardboard", "Food Organics", "Glass", "Metal", "Miscellaneous Trash", "Paper", "Plastic", "Textile Trash", "Vegetation"]


# Create the LabelEncoder instance
label_encoder = LabelEncoder()

# Fit the label encoder with the categories list
label_encoder.fit(CATEGORIES)

# Save the encoder to the same file
file_path = 'label_encoder.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(label_encoder, file)

# Print the encoding result to check
print("Encoded categories:", label_encoder.transform(categories))


Encoded categories: [0 1 2 3 4 5 6 7 8]


In [29]:
import pickle

# Path to your label_encoder.pkl file
file_path = 'label_encoder.pkl'

# Load the LabelEncoder from the file
with open(file_path, 'rb') as file:
    label_encoder = pickle.load(file)

# Now you can use label_encoder to transform or inverse transform categories
print("Classes in the encoder:", label_encoder.classes_)

# Example: Transforming a category to its encoded label
encoded_label = label_encoder.transform(["Plastic"])
print("Encoded label for 'Plastic':", encoded_label)


Classes in the encoder: ['Cardboard' 'Food Organics' 'Glass' 'Metal' 'Miscellaneous Trash' 'Paper'
 'Plastic' 'Textile Trash' 'Vegetation']
Encoded label for 'Plastic': [6]


In [11]:
label_encoder.inverse_transform([6])[0]

'Plastic'

In [21]:
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from flask import Flask, request, jsonify
from PIL import Image

# ============================
# STEP 1: DATA PREPROCESSING
# ============================
DATASET_PATH = "RealWaste"  # Update dataset path

# Ensure this matches the number of categories in your dataset
CATEGORIES = ["glass", "metal", "non_recyclable", "organic", "paper", "plastic", "uncertain", "category8", "category9"]

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  # 80% Training, 20% Validation
    rotation_range=20,
    horizontal_flip=True,
    zoom_range=0.2
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# ============================
# STEP 2: CNN MODEL DEFINITION
# ============================
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(len(CATEGORIES), activation='softmax')  # Output layer now matches CATEGORIES
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ============================
# STEP 3: CHECKING GPU USAGE
# ============================
if tf.test.is_gpu_available():
    print("💪 Using GPU for training.")
else:
    print("❌ No GPU found, using CPU.")

# ============================
# STEP 4: TRAIN THE CNN MODEL
# ============================
model.fit(train_data, epochs=20, validation_data=val_data, verbose=1)

# Save Model
model.save("waste_cnn_model.h5")
print("\n✅ CNN Model Trained & Saved!")

# ============================
# STEP 5: FLASK API FOR PREDICTIONS
# ============================
# app = Flask(__name__)

# Load Trained Model
# cnn_model = tf.keras.models.load_model("waste_cnn_model.h5")

# @app.route('/predict_waste', methods=['POST'])
# def predict_waste():
#     file = request.files['file']
#     img = Image.open(file).resize((224, 224))
#     img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

#     predictions = cnn_model.predict(img_array)
#     predicted_class = CATEGORIES[np.argmax(predictions)]

#     return jsonify({"Predicted Waste Category": predicted_class})

# if __name__ == '__main__':
#     print("\n🚀 Starting Flask Server...\n")
#     app.run(debug=True)


Found 3803 images belonging to 9 classes.
Found 949 images belonging to 9 classes.
❌ No GPU found, using CPU.
Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 124s 997ms/step - accuracy: 0.3269 - loss: 1.9260 - val_accuracy: 0.5490 - val_loss: 1.2751
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 117s 981ms/step - accuracy: 0.6457 - loss: 1.0184 - val_accuracy: 0.5943 - val_loss: 1.1459
Epoch 3/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 116s 974ms/step - accuracy: 0.7132 - loss: 0.8278 - val_accuracy: 0.6080 - val_loss: 1.0850
Epoch 4/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.7388 - loss: 0.7335 - val_accuracy: 0.6017 - val_loss: 1.0772
Epoch 5/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 114s 956ms/step - accuracy: 0.7819 - loss: 0.6435 - val_accuracy: 0.6238 - val_loss: 1.0444
Epoch 6/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 117s 983ms/step - accuracy: 0.7989 - loss: 0.5962 - val_accuracy: 0.6185 - val_loss: 1.0262
Epoch 7/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 116s 978ms/step - accuracy: 0.8187 - loss: 0.5427 - val_accur


✅ CNN Model Trained & Saved!


In [3]:
import cv2
import numpy as np
import os
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from PIL import Image

# ============================
# STEP 6: TEST THE MODEL WITH 10 RANDOM SAMPLES
# ============================

# Load Trained Model
model = load_model("waste_cnn_model.h5")

# Define categories again (for prediction purposes)
CATEGORIES = ["Cardboard", "Food Organics", "Glass", "Metal", "Miscellaneous Trash", "Paper", "Plastic", "Textile Trash", "Vegetation"]

# Dataset path (same as during training)
DATASET_PATH = "RealWaste"

# Function to load and preprocess the image
def load_and_preprocess_image(img_path):
    img = Image.open(img_path).resize((224, 224))  # Resize to the expected input size for MobileNetV2
    img_array = np.array(img) / 255.0  # Rescale the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# List all images in the dataset directory (assuming each class is in its own subdirectory)
image_paths = []
for root, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        if file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file types
            image_paths.append(os.path.join(root, file))

# Pick 10 random images
random_samples = random.sample(image_paths, 10)

# Predict and display the results for 10 random samples
for img_path in random_samples:
    img_array = load_and_preprocess_image(img_path)
    
    # Predict the category for the image
    predictions = model.predict(img_array)
    predicted_class = CATEGORIES[np.argmax(predictions)]
    
    print(f"Image: {img_path}")
    print(f"Predicted Category: {predicted_class}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image: RealWaste\Textile Trash\Textile Trash_283.jpg
Predicted Category: Textile Trash

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Image: RealWaste\Textile Trash\Textile Trash_88.jpg
Predicted Category: Miscellaneous Trash

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Image: RealWaste\Plastic\Plastic_724.jpg
Predicted Category: Plastic

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Image: RealWaste\Miscellaneous Trash\Miscellaneous Trash_58.jpg
Predicted Category: Miscellaneous Trash

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Image: RealWaste\Plastic\Plastic_6.jpg
Predicted Category: Plastic

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Image: RealWaste\Food Organics\Food Organics_102.jpg
Predicted Category: Food Organics

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
Image: RealWaste\Miscellaneous Trash\Miscellaneous Trash_335.jpg
Predicted Category: Miscellaneous Trash

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Image: RealWaste\Vegetation\Vegetation_373.jpg
Predicted Category: Vegeta

In [5]:
def biogas_conversion(mass_vegetation_kg):
    # Constants
    water_ratio = 1.5  # Water to vegetation ratio (1:1.5)
    inoculum_ratio = 0.1  # Inoculum to total slurry ratio (10%)
    biogas_yield = 0.3  # Biogas yield per kg of dry matter (m³/kg)
    dry_matter_content = 0.8  # Dry matter content in vegetation (80%)
    methane_content = 0.6  # Methane content in biogas (60%)
    methane_energy = 35.8  # Energy content of methane (MJ/m³)

    # Step 1: Calculate water and slurry
    water_kg = mass_vegetation_kg * water_ratio
    total_slurry_kg = mass_vegetation_kg + water_kg

    # Step 2: Calculate inoculum
    inoculum_kg = total_slurry_kg * inoculum_ratio

    # Step 3: Calculate dry matter and biogas yield
    dry_matter_kg = mass_vegetation_kg * dry_matter_content
    biogas_volume_m3 = dry_matter_kg * biogas_yield

    # Step 4: Calculate methane volume and energy content
    methane_volume_m3 = biogas_volume_m3 * methane_content
    energy_content_mj = methane_volume_m3 * methane_energy
    energy_content_kwh = energy_content_mj * 0.2778  # Convert MJ to kWh

    # Step 5: Calculate biofuel production (methane as biofuel)
    biofuel_liters = methane_volume_m3  # 1 m³ methane ≈ 1 liter diesel equivalent

    # Output the step-by-step process
    print("\n--- Step-by-Step Process for Converting Vegetation into Biogas ---")
    print(f"1. Input Vegetation: {mass_vegetation_kg} kg")
    print(f"2. Add Water: {water_kg} kg (to create a slurry with a 1:{water_ratio} ratio)")
    print(f"3. Add Inoculum: {inoculum_kg:.1f} kg (10% of total slurry)")
    print(f"4. Total Slurry: {total_slurry_kg:.1f} kg (vegetation + water)")
    print(f"5. Dry Matter in Vegetation: {dry_matter_kg} kg (80% of vegetation)")
    print(f"6. Biogas Yield: {biogas_volume_m3:.1f} m³ (0.3 m³ per kg of dry matter)")
    print(f"7. Methane in Biogas: {methane_volume_m3:.1f} m³ (60% of biogas)")
    print(f"8. Energy Content: {energy_content_mj:.1f} MJ or {energy_content_kwh:.1f} kWh")
    print(f"9. Biofuel Production: {biofuel_liters:.1f} liters of diesel equivalent")

    return {
        "water_kg": water_kg,
        "inoculum_kg": inoculum_kg,
        "total_slurry_kg": total_slurry_kg,
        "dry_matter_kg": dry_matter_kg,
        "biogas_volume_m3": biogas_volume_m3,
        "methane_volume_m3": methane_volume_m3,
        "energy_content_mj": energy_content_mj,
        "energy_content_kwh": energy_content_kwh,
        "biofuel_liters": biofuel_liters,
    }


# User input
mass_vegetation_kg = float(input("Enter the mass of vegetation (kg): "))

results = biogas_conversion(mass_vegetation_kg)

Enter the mass of vegetation (kg):  1000



--- Step-by-Step Process for Converting Vegetation into Biogas ---
1. Input Vegetation: 1000.0 kg
2. Add Water: 1500.0 kg (to create a slurry with a 1:1.5 ratio)
3. Add Inoculum: 250.0 kg (10% of total slurry)
4. Total Slurry: 2500.0 kg (vegetation + water)
5. Dry Matter in Vegetation: 800.0 kg (80% of vegetation)
6. Biogas Yield: 240.0 m³ (0.3 m³ per kg of dry matter)
7. Methane in Biogas: 144.0 m³ (60% of biogas)
8. Energy Content: 5155.2 MJ or 1432.1 kWh
9. Biofuel Production: 144.0 liters of diesel equivalent


In [5]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 9)                   │           4,617 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,918,475 (11.13 MB)

 Trainable params: 660,489 (2.52 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 2 (12.00 B)

In [1]:
def process_material(material_type, mass_kg):
    if material_type == "paper":
        # Paper to Biogas or Bioethanol
        biogas_yield = 0.25  # m³ per kg
        bioethanol_yield = 0.25  # liters per kg
        digestion_time_days = 25  # days
        fermentation_time_days = 6  # days

        biogas_volume = mass_kg * biogas_yield
        bioethanol_volume = mass_kg * bioethanol_yield

        print("\n--- Paper Processing ---")
        print(f"1. Biogas Yield: {biogas_volume:.1f} m³ (anaerobic digestion, {digestion_time_days} days)")
        print(f"2. Bioethanol Yield: {bioethanol_volume:.1f} liters (fermentation, {fermentation_time_days} days)")

    elif material_type == "glass":
        # Glass Recycling
        recycling_time_days = 1.5  # days

        print("\n--- Glass Processing ---")
        print(f"1. Glass can be recycled into new products.")
        print(f"2. Processing Time: {recycling_time_days} days")

    elif material_type == "plastic":
        # Plastic to Bio-oil or Syngas
        biooil_yield = 0.65  # liters per kg
        syngas_yield = 1.25  # m³ per kg
        pyrolysis_time_hours = 1.5  # hours
        gasification_time_hours = 1.5  # hours

        biooil_volume = mass_kg * biooil_yield
        syngas_volume = mass_kg * syngas_yield

        print("\n--- Plastic Processing ---")
        print(f"1. Bio-oil Yield: {biooil_volume:.1f} liters (pyrolysis, {pyrolysis_time_hours} hours)")
        print(f"2. Syngas Yield: {syngas_volume:.1f} m³ (gasification, {gasification_time_hours} hours)")

    elif material_type == "cardboard":
        # Cardboard to Biogas or Bioethanol
        biogas_yield = 0.25  # m³ per kg
        bioethanol_yield = 0.25  # liters per kg
        digestion_time_days = 25  # days
        fermentation_time_days = 6  # days

        biogas_volume = mass_kg * biogas_yield
        bioethanol_volume = mass_kg * bioethanol_yield

        print("\n--- Cardboard Processing ---")
        print(f"1. Biogas Yield: {biogas_volume:.1f} m³ (anaerobic digestion, {digestion_time_days} days)")
        print(f"2. Bioethanol Yield: {bioethanol_volume:.1f} liters (fermentation, {fermentation_time_days} days)")

    elif material_type == "food organics":
        # Food Organics to Biogas or Compost
        biogas_yield = 0.4  # m³ per kg
        compost_yield = 0.6  # kg compost per kg food waste
        digestion_time_days = 25  # days
        composting_time_days = 45  # days

        biogas_volume = mass_kg * biogas_yield
        compost_volume = mass_kg * compost_yield

        print("\n--- Food Organics Processing ---")
        print(f"1. Biogas Yield: {biogas_volume:.1f} m³ (anaerobic digestion, {digestion_time_days} days)")
        print(f"2. Compost Yield: {compost_volume:.1f} kg (composting, {composting_time_days} days)")

    else:
        print("Invalid material type. Please choose 'paper', 'glass', 'plastic', 'cardboard', or 'food organics'.")


# User input
material_type = input("Enter the material type (paper/glass/plastic/cardboard/food organics): ").lower()
mass_kg = float(input("Enter the mass of the material (kg): "))

# Perform calculations and display results
process_material(material_type, mass_kg)

Enter the material type (paper/glass/plastic/cardboard/food organics):  paper
Enter the mass of the material (kg):  1000



--- Paper Processing ---
1. Biogas Yield: 250.0 m³ (anaerobic digestion, 25 days)
2. Bioethanol Yield: 250.0 liters (fermentation, 6 days)
